In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('/content/npc 18 test for colab 2.csv')

In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:11].values
Y = data.iloc[:,11:].values

train_data,test_data,train_targets,test_targets=train_test_split(X,Y,test_size=0.1,random_state=42)

In [ ]:
print(test_data[0])
print(test_targets[0])

ANN modelling using boston method


In [ ]:
#mean = train_data.mean(axis=0)
#train_data -= mean
#std = train_data.std(axis=0)
#train_data /= std
#test_data -= mean
#test_data /= std

In [ ]:
print(test_data[0])

In [ ]:
#build network

from keras import models
from keras import layers

def build_model():
  model=models.Sequential()
  model.add(layers.Dense(6,activation='relu',input_shape=(train_data.shape[1],)))
  #model.add(layers.Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(layers.Dense(1))#,activation='relu'))
  model.compile(optimizer='adam', loss='mse', metrics=['mae'])
  return model

In [ ]:
import numpy as np

k=6
num_val_samples=len(train_data)//k
num_epochs=100
all_scores=[]

for i in range(k):
  print('processing fold#',i)
  val_data=train_data[i*num_val_samples:(i+1)*num_val_samples]
  val_targets=train_targets[i*num_val_samples:(i+1)*num_val_samples]
  partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)

  model=build_model()
  model.fit(partial_train_data,partial_train_targets,epochs=num_epochs,verbose=0)#batch_size=1
  val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=0)
  all_scores.append(val_mae)

In [ ]:
print(all_scores)
print(np.mean(all_scores))

In [ ]:
num_epochs = 2000
all_mae_histories = []
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)
  model = build_model()
  history = model.fit(partial_train_data, partial_train_targets,validation_data=(val_data, val_targets),epochs=num_epochs, batch_size=1, verbose=0)
  mae_history = history.history['val_mae']
  all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor)) 
    else:
      smoothed_points.append(point)
  return smoothed_points
smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
model = build_model()
model.fit(train_data, train_targets,
epochs=1900, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mae_score)

In [ ]:
test=pd.read_csv('/content/test data line.csv')

testdata=test.values
print(testdata)

In [ ]:
prediction=model.predict(testdata)

In [ ]:
print(prediction)

In [ ]:
#test2=pd.read_csv('/content/test data line 2.csv')

#testdata2=test2.values
#print(testdata2)
#prediction=model.predict(testdata2)
#print(prediction)